In [1]:
# data_path_train_orig = '/content/drive/MyDrive/boosters_semrush/data/semrush_cup_train_data.csv'
# data_path_test_orig = '/content/drive/MyDrive/boosters_semrush/data/semrush_cup_test_data.csv'
# data_path_sample_orig = '/content/drive/MyDrive/boosters_semrush/data/semrush_cup_sub.csv'
# data_path_dict_orig = '/content/drive/MyDrive/boosters_semrush/data/semrush_cup_categories_and_demo.csv'

# cache_path = '/content/drive/MyDrive/boosters_semrush/cache'

In [2]:
data_path_train_orig = 'data/semrush_cup_train_data.csv'
data_path_test_orig = 'data/semrush_cup_test_data.csv'
data_path_sample_orig = 'data/semrush_cup_sub.csv'
data_path_dict_orig = 'data/semrush_cup_categories_and_demo.csv'

cache_path = 'cache'

In [3]:
!nvidia-smi

/usr/bin/sh: 1: nvidia-smi: not found


# Init

In [4]:
# !pip3 install sklearn
# !pip install tensorflow 
# !pip3 install tensorflow_addons
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
import pickle
import os
from pathlib import Path
from tqdm.notebook import trange, tqdm

from sklearn.metrics import f1_score  
from sklearn.model_selection import train_test_split

In [6]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers

2022-04-14 15:22:27.702851: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-14 15:22:27.702887: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
def read_csv_cached(csv_path, cache_path):
    data = pd.read_csv(csv_path)
    return data 

#     print(f'read_csv_cached: csv_path={csv_path} cache_path={cache_path}')

#     os.makedirs(cache_path, exist_ok=True)

#     csv_path_cached_pickle = Path(cache_path) / (Path(csv_path).name + '.pickle')
#     print(f'read_csv_cached: csv_path_cached_pickle={csv_path_cached_pickle}')

#     if not(Path(csv_path_cached_pickle).exists()):
#         print(f'read_csv_cached: missing cache')
#         data = pd.read_csv(csv_path)
        
#         with open(csv_path_cached_pickle, 'wb') as f:
#             pickle.dump(data, f)
#     else:
#         print(f'read_csv_cached: cached')

#     print(f'read_csv_cached: reading from cache')
#     with open(csv_path_cached_pickle, 'rb') as f:      
#         data = pickle.load(f)
    
#     print(f'read_csv_cached: done')
#     return data

def split_train(train_path, cache_path):
    print(f'split_train: train_path={train_path} cache_path={cache_path}')

    os.makedirs(cache_path, exist_ok=True)

    data_path_cached_pickle = Path(cache_path) / ('split_train' + '.pickle')
    print(f'split_train: data_path_cached_pickle={data_path_cached_pickle}')

    if not(Path(data_path_cached_pickle).exists()):
        print(f'split_train: missing cache')

        train = read_csv_cached(train_path, cache_path)

        uids_list = train['device_id'].unique()
        train_train_uids, train_test_uids = train_test_split(uids_list, train_size=0.6, random_state=7)
        train_test_uids, train_valid_uids = train_test_split(train_test_uids, train_size=0.5, random_state=7)

        train_train_data = train.loc[train['device_id'].isin(train_train_uids)]
        train_test_data = train.loc[train['device_id'].isin(train_test_uids)]
        train_valid_data = train.loc[train['device_id'].isin(train_valid_uids)]

        data = (train_train_data, train_test_data, train_valid_data)
        
        with open(data_path_cached_pickle, 'wb') as f:
            pickle.dump(data, f)
    else:
        print(f'split_train: cached')

    print(f'split_train: reading from cache')
    with open(data_path_cached_pickle, 'rb') as f:      
        data = pickle.load(f)
    
    train_train, train_test, train_valid = data
    print(f'data size: train={len(train_train)}, test={len(train_test)}, valid={len(train_valid)}')

    return data

# Read data

In [8]:
train_train, train_test, train_valid = split_train(data_path_train_orig, cache_path)

split_train: train_path=data/semrush_cup_train_data.csv cache_path=cache
split_train: data_path_cached_pickle=cache/split_train.pickle
split_train: cached
split_train: reading from cache
data size: train=7810396, test=2632773, valid=2597474


In [9]:
# train_train = train_train[:1100000].copy()
# train_test = train_train[:1100000].copy()

In [10]:
train_train[train_train.event_group_id==39147]

,event_group_id,device_id,timestamp,domain,referrer_domain,url,referrer_url,referrer_num,is_referrer
0,39147,100000,1174857389,ttGJTM.AK,ttNggB.jg,tWAmxZ.ttGJTM.AK/jCYBFH/jlYGps/jjGbCS/jrsypg,tWAmxZ.ttNggB.jg/jdSNxb,0,False
1,39147,100000,1174857499,ttNggB.jg,NaN,tWAmxZ.ttNggB.jg/jHSjLY,NaN,1,False
2,39147,100000,1174857514,ttdgZf.jg,ttNggB.jg,ttdgZf.jg/jdSNxb,tWAmxZ.ttNggB.jg/jdSNxb,2,False
3,39147,100000,1174857523,ttNggB.jg,ttdgZf.jg,tWAmxZ.ttNggB.jg/jHSjLY,ttdgZf.jg/jdSNxb,3,False
4,39147,100000,1174857529,ttsUFm.MZ,ttNggB.jg,ttsUFm.MZ/jjdsdQ/jHiiZc/jHICvz/jjIijp/jdLZBy/j...,tWAmxZ.ttNggB.jg/jdSNxb,4,False
5,39147,100000,1174857546,ttBWMn.jg,ttNggB.jg,tWAmxZ.ttBWMn.jg/jHLvKC/jdSNxb,tWAmxZ.ttNggB.jg/jdSNxb,5,True
6,39147,100000,1174857562,tWxuXo.MZ,ttNggB.jg,ttjvjg.tWxuXo.MZ/jHKZdc/jrbQKv/jjHNIG/jHgCli/j...,tWAmxZ.ttNggB.jg/jdSNxb,6,False
7,39147,100000,1174857665,ttxgCt.jg,ttBWMn.jg,tWAmxZ.ttxgCt.jg/jjxlps,tWAmxZ.ttBWMn.jg/jjGuNS/jjcbKW,7,False
8,39147,100000,1174857670,tWzTqL.jg,ttxgCt.jg,ttuNso.tWzTqL.jg/jvjYYW,tWAmxZ.ttxgCt.jg/jjxlps,8,False
9,39147,100000,1174857671,ttkgWu.jg,tWzTqL.jg,tWAmxZ.ttkgWu.jg/jHcdBZ/jdSNxb,ttuNso.tWzTqL.jg/jvjYYW,9,False


In [11]:
# sample = read_csv_cached(data_path_sample_orig, cache_path)

In [12]:
# sample

In [13]:
train_train.groupby(['device_id','event_group_id']).size().groupby('device_id').size().describe()

count    120000.000000
mean          5.916967
std           9.970268
min           1.000000
25%           1.000000
50%           3.000000
75%           7.000000
max        1047.000000
dtype: float64

In [14]:
train_train.device_id.nunique()

120000

In [15]:
train_train[train_train['device_id']==190706]

,event_group_id,device_id,timestamp,domain,referrer_domain,url,referrer_url,referrer_num,is_referrer
5971163,4418,190706,1175986901,ttxXHJ.jg,ttBzRA.jg,tWAmxZ.ttxXHJ.jg/jjHcfz,tWWFjx.ttBzRA.jg/jdSNxb,0,False
5971164,4418,190706,1175986902,ttNggB.jg,ttxXHJ.jg,ttMMTG.ttNggB.jg/jCCdbb/jjYFWl/jdfcSI,tWAmxZ.ttxXHJ.jg/jjHcfz,1,False
5971165,4418,190706,1175986902,tWxFjf.jg,ttgMnT.jl,tWAmxZ.tWxFjf.jg/jdlNWs/jvyGll,tWAmxZ.ttgMnT.jl/jHgKip/jdSNxb,2,False
5971166,4418,190706,1175986904,ttTNdj.jg,ttzgqo.mp,tWNzCf.ttTNdj.jg/jCBGdF,ttzgqo.mp/jHLllj/jjgdsK/jjKQFK,3,False
5971167,4418,190706,1175986907,ttjLGW.jg,ttTNdj.jg,tWAmxZ.ttjLGW.jg/jdSNxb,tWNzCf.ttTNdj.jg/jCBGdF,4,False
...,...,...,...,...,...,...,...,...,...
5982675,1732934,190706,1168143197,tWvMXx.jg,ttNdzT.jg,ttFxnC.tWvMXx.jg/jdpIGx/jCFZxy/jrvBxy,ttZRCL.ttNdzT.jg/jHYFcp/jvlSWp/jHxjCI/jHfCbu,6,False
5982676,1732934,190706,1168143197,ttdvmg.jg,ttNdzT.jg,ttHoTJ.ttCHdT.ttdvmg.jg/jriFLF/jdZNsy/jCdgri,ttZRCL.ttNdzT.jg/jHYFcp/jvlSWp/jHxjCI/jHfCbu,7,False
5982677,1732934,190706,1168143198,ttWzmJ.jg,NaN,tWMfZg.ttWzmJ.jg/jdxrLy/jCWzXX/jlZHbN,NaN,8,False
5982678,1732934,190706,1168143198,tWxFjf.jg,ttNdzT.jg,ttuVLV.tWxFjf.jg/jCzXYG/jvWYCN/jrfISl/jlggNY,ttZRCL.ttNdzT.jg/jHYFcp/jvlSWp/jHxjCI/jHfCbu,9,False


# Load Domains

In [16]:
print('Loading domains from train.csv')
domains1 = read_csv_cached(data_path_train_orig, cache_path)
domains1 = domains1[['domain']]
domains1['domain'] = domains1['domain'].fillna("EMPTY")
domains1 = domains1.drop_duplicates()
print(f'domains1: {len(domains1)}')

print('Loading domains from test.csv')
domains2 = read_csv_cached(data_path_test_orig, cache_path)
domains2 = domains2[['domain']]
domains2['domain'] = domains2['domain'].fillna("EMPTY")
domains2 = domains2.drop_duplicates()
print(f'domains2: {len(domains2)}')

domains = pd.concat([domains1, domains2])
domains = domains.drop_duplicates()
# domains = domains.sort_values()
domains = domains.reset_index(drop=True)
print(f'domains: {len(domains)}')
del (domains1, domains2)

Loading domains from train.csv
domains1: 947918
Loading domains from test.csv
domains2: 569788
domains: 1212771


In [17]:
# domains[['domain1', 'domain2', 'domain3', 'domain4']] = domains.domain.apply(lambda x: x[::-1]).str.split(".", expand=True).fillna('EMPTY')

domains = domains.set_index('domain')
# for i in range(1,5):
#     print(i)
#     domains[f'domain{i}'] = domains.groupby(f'domain{i}').ngroup()
#     domains = domains.rename(columns={f"domain{i}": f"domain{i}_id"})

domains['domain_id'] = domains.groupby('domain').ngroup()

In [18]:
domains.head()

,domain_id
domain,
ttGJTM.AK,410567
ttNggB.jg,566993
ttdgZf.jg,797192
ttsUFm.MZ,1050429
ttBWMn.jg,331063


In [19]:
domains_dict_id =  {index:value for index, value in domains.domain_id.items()}
# domains1_dict_id = {index:value for index, value in domains.domain1_id.items()}
# domains2_dict_id = {index:value for index, value in domains.domain2_id.items()}
# domains3_dict_id = {index:value for index, value in domains.domain3_id.items()}
# domains4_dict_id = {index:value for index, value in domains.domain4_id.items()}
# domains_dict_id

# Process data

In [20]:
def preprocess_data(data_raw):
    print('Processing history')
    data1 = data_raw[data_raw.referrer_num!='target'].copy()

    data1_domain = data1.domain.fillna('EMPTY').copy()
    data1['domain']  = data1_domain.map(domains_dict_id).astype('int')

    print('. Pivoting history')
    data1 = data1.pivot(index=['device_id', 'event_group_id'], columns='referrer_num', values=['timestamp', 'domain', 'is_referrer'])
    data1.columns = ['_'.join(col).strip() for col in data1.columns.values]

    print('. Updating pivot dtypes')
    data1 = data1.astype('int')
    # print(data1.columns)
    # print(data1.dtypes)         

    print('Processing target')
    data2 = data_raw[data_raw.referrer_num == 'target']
    data2 = data2[['event_group_id', 'timestamp', 'domain']]

    data2_domain = data2.domain.fillna('EMPTY').copy()
    data2['domain']  = data2_domain.map(domains_dict_id).astype('int')
    data2.columns = data2.columns

    data2 = data2.set_index('event_group_id')    
    # print(data2.columns)
    # print(data2.dtypes)     

    print('. Merging results')
    data_new = data2.join(data1)
    data_new.loc[:,data_new.columns.str.startswith('timestamp_')] = np.repeat(np.expand_dims(data_new['timestamp'].values, axis=1), 10, axis=1) - data_new.loc[:,data_new.columns.str.startswith('timestamp_')]
    # data_new.iloc[:,1:11] = np.repeat(np.expand_dims(data_new['timestamp'].values, axis=1), 10, axis=1) - data_new.iloc[:,1:11].values
    data_new = data_new.drop(columns='timestamp')
    # print(data_new.columns)
    # print(data_new.dtypes)    

    return data_new

In [21]:
train_train = preprocess_data(train_train)
train_test = preprocess_data(train_test)
# train_valid = preprocess_data(train_valid)

Processing history
. Pivoting history
. Updating pivot dtypes
Processing target
. Merging results
Processing history
. Pivoting history
. Updating pivot dtypes
Processing target
. Merging results


In [22]:
train_train.columns

Index(['domain', 'timestamp_0', 'timestamp_1', 'timestamp_2', 'timestamp_3',
       'timestamp_4', 'timestamp_5', 'timestamp_6', 'timestamp_7',
       'timestamp_8', 'timestamp_9', 'domain_0', 'domain_1', 'domain_2',
       'domain_3', 'domain_4', 'domain_5', 'domain_6', 'domain_7', 'domain_8',
       'domain_9', 'is_referrer_0', 'is_referrer_1', 'is_referrer_2',
       'is_referrer_3', 'is_referrer_4', 'is_referrer_5', 'is_referrer_6',
       'is_referrer_7', 'is_referrer_8', 'is_referrer_9'],
      dtype='object')

# Get Dataset

In [23]:
def get_raw_dataset(data):
    # Domain
    domain = data[['domain']]
    domain = [value.values for key, value in domain.groupby('device_id')]
    # print(domain[0].shape)
    domain_dataset = tf.data.Dataset.from_generator(lambda: domain,
                                                    domain[0].dtype,
                                                    tf.TensorShape([None, domain[0].shape[1]]))   
    
    # Timestamp history
    timestamp_history = data.loc[:,data.columns.str.startswith('timestamp_')]
    timestamp_history = [value.values for key, value in timestamp_history.groupby('device_id')]
    timestamp_history_dataset = tf.data.Dataset.from_generator(lambda: timestamp_history,
                                                               timestamp_history[0].dtype,
                                                               tf.TensorShape([None, timestamp_history[0].shape[1]]))    
    
    # Domain history
    domain_history = data.loc[:,data.columns.str.startswith('domain_')]
    domain_history = [value.values for key, value in domain_history.groupby('device_id')]
    # print(domain_history[0].shape)
    domain_history_dataset = tf.data.Dataset.from_generator(lambda: domain_history,
                                                               domain_history[0].dtype,
                                                               tf.TensorShape([None, domain_history[0].shape[1]])) 
    # Target
    target = data.loc[:,data.columns.str.startswith('is_referrer_')]
    target = [value.values for key, value in target.groupby('device_id')]
    target_dataset = tf.data.Dataset.from_generator(lambda: target,
                                                    target[0].dtype,
                                                    tf.TensorShape([None, target[0].shape[1]]))
    
#     timeframe_history = tf.data.Dataset.from_tensor_slices(np.expand_dims(data.iloc[:,:10].values, axis=1))
#     domain_history = tf.data.Dataset.from_tensor_slices(np.expand_dims(data.iloc[:,10:20].values, axis=1))


#     domain = tf.data.Dataset.from_tensor_slices(data.domain.values)    
#     labels = tf.data.Dataset.from_tensor_slices(data.iloc[:,20:30].values)

#     features_dataset = tf.data.Dataset.zip((timeframe_history, domain_history, 
#                                             domain1_history, domain2_history, domain3_history, domain4_history,
#                                             domain))
    features_dataset = tf.data.Dataset.zip((domain_dataset, timestamp_history_dataset, domain_history_dataset))
    dataset = tf.data.Dataset.zip((features_dataset, target_dataset))
    return dataset

def get_train_dataset(data):
    dataset = get_raw_dataset(data)
    dataset = dataset.shuffle(64*20)    
    dataset = dataset.batch(1)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)   

    return dataset

def get_test_dataset(data):
    dataset = get_raw_dataset(data)
    dataset = dataset.batch(1)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

# train_train_dataset = get_train_dataset(train_train)
# train_test_dataset = get_test_dataset(train_test)

# for i in train_train_dataset.take(2):
#     print(i)

# Create model

In [24]:
group_domain_input = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='group_domain')
group_timestamp_history_input = tf.keras.layers.Input(shape=(None, 10), dtype=tf.int32, name='group_timestamp_history')
group_domain_history_input = tf.keras.layers.Input(shape=(None, 10), dtype=tf.int32, name='group_domain_history')

domain_embedding = tf.keras.layers.Embedding(input_dim=len(domains_dict_id), output_dim=128)

group_domain = group_domain_input
group_domain = domain_embedding(group_domain)
# print(group_domain.shape)

timestamp_history = group_timestamp_history_input
timestamp_history = tf.cast(timestamp_history, tf.float32) / 1000
timestamp_history = tf.expand_dims(timestamp_history, 3)
print(timestamp_history.shape)

group_domain_history = group_domain_history_input
group_domain_history = domain_embedding(group_domain_history)
print(group_domain_history.shape)

group_history = tf.keras.layers.Concatenate()([timestamp_history, group_domain_history])

group_domain_history = tf.keras.layers.TimeDistributed(tf.keras.layers.GRU(256, return_sequences=True))(group_history)
group_domain_history = tf.keras.layers.TimeDistributed(tf.keras.layers.GRU(128, return_sequences=False))(group_history)
# group_domain_history = tf.keras.layers.GRU(128, return_sequences=False)(group_domain_history)

group_output = tf.keras.layers.Concatenate()([group_domain_history, group_domain])

group_output = tf.keras.layers.Dense(64, activation='relu')(group_output)
group_output = tf.keras.layers.Dense(10, activation='softmax')(group_output)

print(group_output.shape)
model = tf.keras.Model(inputs=(group_domain_input,
                               group_timestamp_history_input,
                               group_domain_history_input
                              ),
                       outputs=group_output)
model.summary()
# # predicts = [model(x[0]) for x in train_train_dataset.take(2)]
# # predicts
# # model.predict(train_train_dataset.take(2))

2022-04-14 15:23:36.769943: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-14 15:23:36.769974: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-14 15:23:36.770000: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2022-04-14 15:23:36.770197: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, None, 10, 1)
(None, None, 10, 128)
(None, None, 10)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 group_timestamp_history (Input  [(None, None, 10)]  0           []                               
 Layer)                                                                                           
                                                                                                  
 tf.cast (TFOpLambda)           (None, None, 10)     0           ['group_timestamp_history[0][0]']
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, None, 10)     0           ['tf.cast[0][0]']                
                                                                                                  
 group_domain_history (InputLay  [(

In [25]:
# domain_embedding = tf.keras.layers.Embedding(input_dim=len(domains_dict_id), output_dim=8)

# # domain_input = tf.keras.layers.Input(shape=(None, ), dtype=tf.int32, name='domain')
# # print(f'domain_input           ={domain_input.shape}')
# # domain = domain_embedding(domain_input)
# # print(f'domain                 ={domain.shape}')

# # group_timestamp_history_input = tf.keras.layers.Input(shape=(None,10), dtype=tf.int32, name='timestamp_history')
# # group_domain_history_input = tf.keras.layers.Input(shape=(None, 10), dtype=tf.int32, name='domain_history')

# #########################
# # Process each group
# #########################
# # timestamp_history_input = tf.keras.layers.Input(shape=(10), dtype=tf.int32, name='timestamp_history')
# # print(f'timestamp_history_input={timestamp_history_input.shape}')
# # timestamp_history = timestamp_history_input
# # timestamp_history = tf.expand_dims(timestamp_history, 1)
# # timestamp_history = tf.cast(timestamp_history, tf.float32) / 1000
# # timestamp_history = tf.expand_dims(timestamp_history, 3)
# # print(f'timestamp_history      ={timestamp_history.shape}')

# domain_history_input = tf.keras.layers.Input(shape=(10), dtype=tf.int32, name='domain_history')
# print(f'domain_history_input   ={domain_history_input.shape}')
# domain_history = domain_history_input
# domain_history = tf.expand_dims(domain_history, 1)
# domain_history = domain_embedding(domain_history)
# print(f'domain_history         ={domain_history.shape}')

# history = domain_history
# # history = tf.keras.layers.Concatenate()([timestamp_history, domain_history])
# print(f'history                ={history.shape}')

# history = tf.squeeze(history, 1)
# history = layers.SpatialDropout1D(0.05)(history)

# history = tf.keras.layers.GRU(256, return_sequences=True)(history)
# history = tf.keras.layers.GRU(128, return_sequences=False)(history)

# output = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.L1L2(1e-7, 1e-5))(history)
# # output = tf.keras.layers.Dense(10, activation='softmax')(output)
# print(f'output                 ={output.shape}')

# model_by_group = tf.keras.Model(inputs=domain_history_input, outputs=output)
# # model_by_group_layer = tf.keras.layers.Lambda(lambda x: model_by_group((x[0], x[1]), training=True))
# # model_by_group.summary()
# # model_by_group.compile(
# #     loss=keras.losses.CategoricalCrossentropy(),
# #     optimizer=tf.keras.optimizers.Adam(0.001),
# #   #  metrics=tfa.metrics.F1Score(num_classes=10, average='macro')
# # )
# #########################

# # group_domain_input = tf.keras.layers.Input(shape=(None, ), dtype=tf.int32, name='group_domain')
# group_timestamp_history_input = tf.keras.layers.Input(shape=(None, 10), dtype=tf.int32, name='group_timestamp_history')
# group_domain_history_input = tf.keras.layers.Input(shape=(None, 10), dtype=tf.int32, name='group_domain_history')

# group_output = tf.keras.layers.TimeDistributed(model_by_group)(group_domain_history_input)
# group_output = tf.keras.layers.Dense(128, activation='relu')(group_output)
# group_output = tf.keras.layers.Dense(64, activation='relu')(group_output)
# group_output = tf.keras.layers.Dense(10, activation='softmax')(group_output)

# print(group_output.shape)
# model = tf.keras.Model(inputs=(group_timestamp_history_input, group_domain_history_input), outputs=group_output)
# model.summary()
# # predicts = [model(x[0]) for x in train_train_dataset.take(2)]
# # predicts
# # model.predict(train_train_dataset.take(2))

In [26]:
model.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
  #  metrics=tfa.metrics.F1Score(num_classes=10, average='macro')
)

# Fit the model

In [ ]:
train_train_dataset = get_train_dataset(train_train)
train_test_dataset = get_test_dataset(train_test)

train_history = model.fit(train_train_dataset,
                          validation_data=train_test_dataset,
                          epochs=1)

   1191/Unknown - 2343s 2s/step - loss: 1.5731

In [ ]:
train_test_fact = train_test.loc[:,train_test.columns.str.startswith('is_referrer_')].values

In [ ]:
train_test_pred = [model(x[0]) for x in tqdm(train_test_dataset)]

In [ ]:
train_test_pred = tf.concat(train_test_pred, axis=1)

In [ ]:
train_test_pred = tf.squeeze(train_test_pred)

In [ ]:
train_test_pred

In [ ]:
th_list = np.arange(0.1, 0.51, 0.05)
f1_list = []
for th in th_list:
    f1_list.append(f1_score(train_test_fact, train_test_pred >= th, average='macro'))

pd.DataFrame({'th':th_list, 'f1':f1_list}).sort_values('f1', ascending=False)

In [ ]:
train_test_fact_v2 = np.argmax(train_test_fact, axis=1)
train_test_fact_v2

train_test_pred_v2 = np.argmax(train_test_pred, axis=1)
train_test_pred_v2

f1_score(train_test_fact_v2, train_test_pred_v2, average='macro')

In [ ]:
# train_test_pred = model.predict(train_test_dataset, verbose=1)

In [ ]:
train_test_fact = train_test.iloc[:,20:30].values

In [ ]:
th_list = np.arange(0.1, 0.51, 0.05)
f1_list = []
for th in th_list:
    f1_list.append(f1_score(train_test_fact, train_test_pred >= th, average='macro'))

pd.DataFrame({'th':th_list, 'f1':f1_list}).sort_values('f1', ascending=False)

In [ ]:
train_test_pred_max = np.zeros_like(train_test_pred)
arg_max_result = np.argmax(train_test_pred, axis=1)
for i,j in enumerate(arg_max_result):
    train_test_pred_max[i][j] = 1
f1_score(train_test_fact, train_test_pred_max, average='macro')

In [ ]:
train_test_fact_v2 = np.argmax(train_test_fact, axis=1)
train_test_fact_v2

train_test_pred_v2 = np.argmax(train_test_pred, axis=1)
train_test_pred_v2

f1_score(train_test_fact_v2, train_test_pred_v2, average='macro')

# Validate on train_valid

In [ ]:
train_valid_dataset = get_test_dataset(train_valid)
train_valid_pred = model.predict(train_valid_dataset, verbose=1)
train_valid_fact = train_valid.iloc[:,20:30].values

In [ ]:
th_list = np.arange(0.1, 0.51, 0.05)
f1_list = []
for th in th_list:
    f1_list.append(f1_score(train_valid_fact, train_valid_pred >= th, average='macro'))

pd.DataFrame({'th':th_list, 'f1':f1_list}).sort_values('f1', ascending=False)

# Prediction

In [ ]:
test = read_csv_cached(data_path_test_orig, cache_path)

In [ ]:
test = preprocess_data(test, domains_dict_id)